## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key

from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")

# Remove unnecessary extra index column
new_order = ["City", "Country", "Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed", "Current Description"]
city_data_df = city_data_df[new_order]

# Formatting - this broke the code
city_data_df["Lat"] = city_data_df["Lat"].map("{:.2f}".format).astype(float)
city_data_df["Lng"] = city_data_df["Lng"].map("{:.2f}".format).astype(float)

city_data_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Ponta Do Sol,PT,32.67,-17.10,72.23,73,2,4.83,clear sky
1,Ushuaia,AR,-54.80,-68.30,37.02,93,100,27.63,sleet
2,Avarua,CK,-21.21,-159.78,73.45,64,100,9.22,overcast clouds
3,Pangkalanbuun,ID,-2.68,111.62,72.77,99,91,2.28,overcast clouds
4,Fort Nelson,CA,58.81,-122.70,67.62,49,100,11.56,light rain


In [3]:
city_data_df.dtypes
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature (°f) you would like for your trip? "))
max_temp = float(input("What is the maximum temperature (°f) you would like for your trip? "))

In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]

In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City                   0
Country                3
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df.replace("", np.nan)
clean_df = preferred_cities_df.dropna()
print(len(preferred_cities_df))
print(len(clean_df))


244
241


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Ponta Do Sol,PT,72.23,clear sky,32.67,-17.10,
2,Avarua,CK,73.45,overcast clouds,-21.21,-159.78,
3,Pangkalanbuun,ID,72.77,overcast clouds,-2.68,111.62,
4,Fort Nelson,CA,67.62,light rain,58.81,-122.70,
5,Morondava,MG,71.89,clear sky,-20.28,44.28,
...,...,...,...,...,...,...,...
690,Piranshahr,IR,69.75,clear sky,36.70,45.14,
697,Gull Lake,CA,72.54,clear sky,50.10,-108.48,
698,Fushun,CN,69.51,broken clouds,41.86,123.92,
700,Hereford,GB,68.05,overcast clouds,52.06,-2.71,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
    params["location"] = f"{lat},{lng}"
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.") 

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df['Hotel Name'] = hotel_df['Hotel Name'].replace('', np.nan)
hotel_df.dropna()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Ponta Do Sol,PT,72.23,clear sky,32.67,-17.10,Hotel do Campo
2,Avarua,CK,73.45,overcast clouds,-21.21,-159.78,Paradise Inn
3,Pangkalanbuun,ID,72.77,overcast clouds,-2.68,111.62,Grand Kecubung Hotel
4,Fort Nelson,CA,67.62,light rain,58.81,-122.70,The Fort Nelson Hotel
5,Morondava,MG,71.89,clear sky,-20.28,44.28,La Case Bambou
...,...,...,...,...,...,...,...
690,Piranshahr,IR,69.75,clear sky,36.70,45.14,مسافر خانه سعدی
697,Gull Lake,CA,72.54,clear sky,50.10,-108.48,Campbell's Accommodations
698,Fushun,CN,69.51,broken clouds,41.86,123.92,Love Theme Apartment Xinhua Branch
700,Hereford,GB,68.05,overcast clouds,52.06,-2.71,The Green Dragon Hotel


In [10]:
# 8a. Create the output File (CSV)
clean_hotel_df = hotel_df
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv

clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt><b>Hotel Name</b></dt><dd>{Hotel Name}</dd>
<dt><b>City </b></dt><dd>{City}</dd>
<dt><b>Country</b></dt><dd>{Country}</dd>
<dt><b>Current Weather</b></dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))